In [234]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [368]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

## Get Indeks Link

In [55]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    if cat_link == 'otomotif':
        url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    else :
        url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.findAll('div', class_="latest--indeks mt2 clearfix")
    for post in contentDiv:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page = soup.find('div', class_="paging__wrap clearfix")
    if el_page:
        a_page = soup.find('div', class_="paging__wrap clearfix").findAll('div', class_='paging__item')[-1].find('a')
        if 'data-ci-pagination-page' in a_page:
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

## Get Detail Berita

In [447]:
def getDetailBerita(url):
    articles = {}
    #for link in links:
    #link
    #articles['url'] = link[0]
    response = requests.get(url)
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")

    #articleid
    articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])

    #category
    #         articles['category'] = link[1]
    articles['url'] = url

    #extract subcategory from breadcrumb
    bc = soup.find('ul', class_="breadcrumb__wrap")
    articles['subcategory'] = bc.findAll('li')[2].text

    #article
    article = soup.find("div", class_="read__content")

    #extract date
    pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
    pubdate = pubdate.strip(' \t\n\r')
    articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")

    #extract author
    articles['author'] = soup.find('div', class_="read__author").text

    #extract title
    articles['title'] = soup.find('h1', class_="read__title").text      

    #source
    articles['source'] = 'kompas'

    #extract comments count
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')  # Last I checked this was necessary.
    options.add_argument('--disable-extensions')

    driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html5lib')

    comment = soup.find('div', class_="span4 comments-count tright")
    comment_num = comment.text.replace('Ada ', '')
    comment_num = comment_num.replace(' komentar untuk artikel ini', '')
    articles['comments'] = int(comment_num)

    #extract tags
    tags = soup.find("meta", attrs={'name':'content_tag'})['content']
    articles['tags'] = tags

    #extract images
    articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']


    #hapus link sisip
    #         for link in detail.findAll('table', class_="linksisip"):
    #             link.decompose()

    #hapus video sisip
    #         for tag in detail.findAll('div', class_="detail_tag"):
    #             tag.decompose()

    #hapus all setelah clear fix
#     for det in detail.find('div', class_="wfull fl rl"):
#         det.decompose()

    #hapus all script
    for script in detail.findAll('script'):
        script.decompose()
        
    #extract content
    detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
    content = re.sub(r'\n|\t|\b|\r','',detail.text)
#     articles['content'] = re.sub('google*','', content).strip(' ')
    articles['content']
    print('memasukkan berita id ', articles['id'])

#     all_articles.append(articles)

    return articles

In [442]:
url = "https://regional.kompas.com/read/2018/07/28/22524061/sebelum-ott-kpk-lampung-selatan-tiga-kali-dapat-penilaian-zona-merah"

In [374]:
articles = {}
response = requests.get(url)
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")
# soup


In [448]:
detail = [getDetailBerita(url)]
detail

UnboundLocalError: local variable 'detail' referenced before assignment

In [444]:
df = pd.DataFrame(detail)

In [445]:
df

,author,comments,content,id,images,pubdate,source,subcategory,tags,title,url
0,"Kontributor Lampung, Eni Muslihah",7,"BANDAR LAMPUNG, KOMPAS.com - Ombudsman RI Per...",1634180,https://asset.kompas.com/crop/0x0:1000x667/780...,2018-07-28 22:52:40,kompas,Regional,"KPK, Ombudsman, Bandar Lampung, Lampung Selata...","Sebelum OTT KPK, Lampung Selatan Tiga Kali Dap...",https://regional.kompas.com/read/2018/07/28/22...


In [446]:
soup.findAll('a', class_="inner-link")

[<a class="inner-link" href="http://indeks.kompas.com/tag/Bandar-Lampung" style="color:#428bca;" target="_blank">BANDAR LAMPUNG</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Ombudsman" style="color:#428bca;" target="_blank">Ombudsman</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Lampung-Selatan" style="color:#428bca;" target="_blank">Lampung Selatan</a>,
 <a class="inner-link" href="http://nasional.kompas.com/read/2018/07/28/18401441/kasus-suap-bupati-lampung-selatan-kpk-geledah-lima-tempat" target="_blank">Kasus Suap Bupati Lampung Selatan, KPK Geledah Lima Tempat</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/OTT" style="color:#428bca;" target="_blank">OTT</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/KPK" style="color:#428bca;" target="_blank">KPK</a>,
 <a class="inner-link" href="http://indeks.kompas.com/tag/Zainuddin-Hasan" style="color:#428bca;" target="_blank">Zainuddin Hasan</a>,
 <a class="inner-link" href="htt

In [438]:
# detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
# content = re.sub(r'\n|\t|\b|\r','',detail.text)
# articles['content'] = re.sub('google.*','', content).strip(' ')
# articles['content']
# # print('memasukkan berita id ', articles['id'])

In [550]:
base_url = 'https://indeks.kompas.com/2018-07-21'
url = 'http://indeks.kompas.com/news/2018-07-21/2'
html = urlopen(url)
soup = BeautifulSoup(html, 'html5lib')

In [551]:
# soup.findAll('div', class_ = 'latest--indeks mt2 clearfix')

In [541]:
soup.find('h3').a['href']

'http://regional.kompas.com/read/2018/07/21/23072111/pemkot-tasik-dorong-realisasi-pembangunan-tol-cileunyi-tasikmalaya'

In [552]:
soup.find('a', class_='paging__link paging__link--next', href=True)

<a class="paging__link paging__link--next" data-ci-pagination-page="3" href="http://indeks.kompas.com/news/2018-07-21/3" rel="next">Next</a>

In [671]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

In [673]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

def parse():
    base_url = 'https://indeks/kompas.com'
    url = 'http://indeks/kompas.com/news/2018-07-28'

    while True:
        html = urlopen(url)
        soup = BeautifulSoup(html, 'html5lib')

        for article in soup.findAll('div', class_ = 'article__list clearfix'):
            try:    
                print(article.find('h3').a['href'])
            except HTTPError as e:
                print(e)
            except URLError as e:
                print("Server down or incorrect domain")
            except urlopen as e:
                print("Server down or incorrect domain")
            else:
                res = BeautifulSoup(html.read(),"html5lib")
                print(res.titles)

        next_button = soup.find('a', class_='paging__link paging__link--next', href=True)

        if next_button:
            url = base_url + next_button['href']
        else:
            break
parse()

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [602]:
base_url = 'http://rl.odessa.ua/index.php'
url = 'http://rl.odessa.ua/index.php/ru/poslednie-novosti?start=0'

html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [603]:
base_url + next_button['href']

'http://rl.odessa.ua/index.phphttp://indeks.kompas.com/news/2018-07-21/3'

In [666]:
def parse():
    base_url = 'http://indeks.kompas.com'
    url = 'http://indeks.kompas.com/news/2018-07-28'

    while True:
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')

        for article in soup.findAll('div', class_ = 'article__list clearfix'):
            try:    
                print(article.find('h3').a['href'])
            except AttributeError as e:
                print(e)
                
        next_button = soup.find('a', class_='paging__link paging__link--next', href=True)

        if next_button:
            url = base_url + next_button['href']
        else:
            break
            
parse() 

http://regional.kompas.com/read/2018/07/28/23173601/1-agustus-api-obor-asian-games-2018-mampir-ke-danau-toba
http://nasional.kompas.com/read/2018/07/28/23103901/bertemu-demokrat-pks-akan-bahas-opsi-capres-cawapres-alternatif
http://regional.kompas.com/read/2018/07/28/23083031/karir-politik-bupati-lampung-selatan-sebelum-terjegal-fee-rp-700-juta
http://regional.kompas.com/read/2018/07/28/22524061/sebelum-ott-kpk-lampung-selatan-tiga-kali-dapat-penilaian-zona-merah
http://regional.kompas.com/read/2018/07/28/22505251/tilang-on-the-spot-segera-diterapkan-di-surabaya
http://regional.kompas.com/read/2018/07/28/22453671/di-karawang-aksi-long-march-laskar-santri-disambut-pawai-obor
http://regional.kompas.com/read/2018/07/28/22255841/sebelum-dikirim-ilegal-ke-malaysia-31-wna-bangladesh-disembunyikan-di-hutan
http://megapolitan.kompas.com/read/2018/07/28/22232341/syarif-hasil-pemeriksaan-kasn-kok-disampaikan-lewat-siaran-pers
http://nasional.kompas.com/read/2018/07/28/22162421/selamat-dari-kapal

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [584]:
base_url = 'http://rl.odessa.ua/index.php'
url = 'http://rl.odessa.ua/index.php/ru/poslednie-novosti?start=0'

html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

In [562]:
# article = soup.findAll('article', class_ = 'item')
# article

In [530]:
soup.find('h1').find('a').get_text()

'Тренировка перед стартом'

In [532]:
soup.find('p').get_text()

'Еще свежи воспоминания о прошедшей Экологической экспедиции (такое, поверьте, не скоро забудется!), а непоседливые ришельевцы снова готовятся в путь. 24 июля начинается Велосипедная экспедиция Ришельевского лицея по замкам и крепостям Западной Украины. ВЭРЛ-2018 требует серьезной подготовки и проверенного снаряжения. Поэтому перед стартом необходим тренировочный "пробный" поход.\xa0'

In [657]:
# base_url = 'http://indeks.kompas.com/news'
# url = 'http://indeks.kompas.com/news/2018-07-28/1'

# while True:
#     html = urlopen(url)
#     soup = BeautifulSoup(html, 'lxml')

#     for article in soup.findAll('div', class_ = 'article__list clearfix'):
#         try:    
#             print('\t' + article.find('h3').a['href'])
#         except AttributeError as e:
#             print(e)
#     next_button = soup.find('a', class_='paging__link paging__link--next', href=True)

#     if next_button:
#         url = base_url + next_button['href']
#     else:
#         break
    
# #     if status_code == 404: # break once the page is not found
# #         break

In [594]:
next_button = soup.find('a', class_='paging__link paging__link--next', href=True)

In [595]:
next_button['href']

'http://indeks.kompas.com/news/2018-07-21/3'

In [596]:
base_url + next_button['href']

'http://indeks.kompas.com/news/2018-07-21/3'

In [607]:
target_url.format(page_num)

'http://ratings.food.gov.uk/enhanced-search/en-GB/%5E/London/Relevance/0/%5E/%5E/0/5/10'

In [609]:
url = "https://indeks.kompas.com/news/2018-07-07"

In [610]:
url.format(page_num)

'https://indeks.kompas.com/news/2018-07-07'

In [629]:
import requests
from bs4 import BeautifulSoup

target_url = url = "https://indeks.kompas.com/news/2018-07-28"

page_num = 1
while True:
    response = requests.get(target_url.format(page_num))
    if response.status_code == 404: # break once the page is not found
        break
    print("Scraping Page Number {}".format(page_num))
    soup = BeautifulSoup(response.text,"lxml")
    for article in soup.findAll('div', class_ = 'latest--indeks mt2 clearfix'):
        print('\t' + article.find('h3').a['href'])

    page_num += 1

Scraping Page Number 1
	http://regional.kompas.com/read/2018/07/28/23173601/1-agustus-api-obor-asian-games-2018-mampir-ke-danau-toba
Scraping Page Number 2
	http://regional.kompas.com/read/2018/07/28/23173601/1-agustus-api-obor-asian-games-2018-mampir-ke-danau-toba


ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))